In [1]:

%load_ext sql

import pandas as pd
import sqlite3
%sql sqlite:///Course_DW.db 

'Connected: @Course_DW.db'

In [2]:
%%sql
ATTACH DATABASE 'CourseData.db' AS cd

 * sqlite:///Course_DW.db
Done.


[]

In [3]:
%%sql
SELECT *
FROM cd.PROGRAM
LIMIT 5

 * sqlite:///Course_DW.db
Done.


ProgramID,ProgramCode,ProgramName
1,None,None
2,AC,Accounting
3,AE,Applied Ethics
4,AH,Art History
5,AN,Asian Studies


In [4]:
%%sql

DROP TABLE IF EXISTS LOCATION1;
CREATE TABLE LOCATION1 (
    LocationID INTEGER PRIMARY KEY,
    Location_Code TEXT,
    Building TEXT,
    Room TEXT
);

DROP TABLE IF EXISTS INSTRUCTOR1;
CREATE TABLE INSTRUCTOR1 (
    InstructorID INTEGER PRIMARY KEY,
    InstructorName TEXT 
);

DROP TABLE IF EXISTS PROGRAM1;
CREATE TABLE PROGRAM1 (
    ProgramID INTEGER PRIMARY KEY,
    ProgramCode TEXT,
    ProgramName TEXT
);

DROP TABLE IF EXISTS COURSE1;
CREATE TABLE COURSE1 (
    CatalogID TEXT,
    CourseID INTEGER PRIMARY KEY,
    ProgramID INTEGER NOT NULL,
    CourseTitle TEXT NOT NULL,
    Description TEXT,
    Attributes TEXT,
    Coreqs TEXT,
    Prereqs TEXT,
    Fees TEXT,
    Credits INTEGER NOT NULL,
    FOREIGN KEY (ProgramID) REFERENCES PROGRAM (ProgramID)
);

DROP TABLE IF EXISTS COURSE_MEETING1;
CREATE TABLE COURSE_MEETING1 (
    CourseMeetingID INTEGER PRIMARY KEY,
    CatalogID TEXT,
    Location TEXT,
    StartDateTime TEXT,
    EndDateTime TEXT,
    FOREIGN KEY (Location) REFERENCES LOCATION (Location)
);

 * sqlite:///Course_DW.db
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.


[]

In [6]:
%%sql
DELETE FROM LOCATION1;
INSERT INTO LOCATION1 (Location_Code, Building, Room)
SELECT Location_Code, Building, Room
FROM cd.LOCATION;

 * sqlite:///Course_DW.db
0 rows affected.
207 rows affected.


[]

In [9]:
%%sql
DELETE FROM INSTRUCTOR1;

INSERT INTO INSTRUCTOR1 (InstructorName)
SELECT DISTINCT InstructorName
FROM cd.INSTRUCTOR;

 * sqlite:///Course_DW.db
0 rows affected.
1048 rows affected.


[]

In [11]:
%%sql
DELETE FROM PROGRAM1;

INSERT INTO PROGRAM1  (ProgramCode, ProgramName)
SELECT DISTINCT ProgramCode, ProgramName
FROM cd.PROGRAM;

 * sqlite:///Course_DW.db
0 rows affected.
81 rows affected.


[]

In [22]:
%%sql
DELETE FROM COURSE1;

INSERT INTO COURSE1 (CatalogID, ProgramID, CourseTitle, Description, Attributes, Coreqs, Prereqs, Fees, Credits)
SELECT DISTINCT CatalogID, COURSE.ProgramID, CourseTitle, Description, Attributes, Coreqs, Prereqs, Fees, Credits
FROM cd.COURSE

 * sqlite:///Course_DW.db
0 rows affected.
1972 rows affected.


[]

In [26]:
%%sql
DELETE FROM COURSE_MEETING1;

INSERT INTO COURSE_MEETING1 (CatalogID, Location, StartDateTime, EndDateTime)
SELECT DISTINCT CatalogID, Location, StartDateTime, EndDateTime
FROM cd.COURSE_MEETING

 * sqlite:///Course_DW.db
0 rows affected.
280529 rows affected.


[]

In [27]:
%%sql
--FACT TABLE

DROP TABLE IF EXISTS COURSE_OFFERING1;
CREATE TABLE COURSE_OFFERING1 (
    CourseOfferingID INTEGER PRIMARY KEY,
    CatalogID TEXT NOT NULL,
    CourseTitle TEXT NOT NULL,
    InstructorName TEXT NOT NULL,
    Section TEXT, 
    Meetings TEXT NOT NULL,
    Timecodes TEXT NOT NULL,
    Term TEXT NOT NULL,
    StartDay TEXT NOT NULL,
    End TEXT NOT NULL,    
    Cap INTEGER NOT NULL,
    Act INTEGER NOT NULL,
    Rem INTEGER NOT NULL,
    FOREIGN KEY (CatalogID) REFERENCES COURSE_MEETING1 (CatalogID)
    FOREIGN KEY (InstructorName) REFERENCES INSTRUCTOR1 
    (InstructorName)
    FOREIGN KEY (CourseTitle) REFERENCES COURSE1 (CourseTitle)
);



 * sqlite:///Course_DW.db
Done.
Done.


[]

In [ ]:
%%sql
DELETE FROM CLASS_STATISTICS;
INSERT INTO CLASS_STATISTICS (Term_id, Instructor_id, Program_id, Course_id, Location_id, crn, Cap, Act, Rem, Meeting_times, Meeting_hours)
    SELECT DISTINCT TERM1.Term_id, coursedata.CLASS.Instructor_id, coursedata.COURSE.Program_id, coursedata.COURSE.Course_id,
            coursedata.MEETING.Location_id,  crn, Cap, Act, Rem,
        
            COUNT(coursedata.MEETING.Class_id) AS Meeting_times, 
           sum(strftime('%H:%M',coursedata.MEETING.End) - strftime('%H:%M',coursedata.MEETING.Start))AS Meeting_hours
        
    FROM coursedata.CLASS
        LEFT JOIN coursedata.COURSE ON (coursedata.COURSE.CatalogID = coursedata.CLASS.CatalogID)
        LEFT JOIN coursedata.MEETING ON (coursedata.CLASS.Class_id = coursedata.MEETING.Class_id)
        LEFT JOIN TERM1 USING(Term)
        WHERE coursedata.COURSE.Catalog_year <> '2018_2019'
    GROUP BY coursedata.MEETING.Class_id

In [39]:
%%sql
DELETE FROM COURSE_OFFERING1;

INSERT INTO COURSE_OFFERING1 (CatalogID, CourseTitle, InstructorName, Section, Meetings, TimeCodes, Term, StartDay, End, Cap, Act, Rem)
SELECT DISTINCT cd.COURSE.CatalogID, cd.COURSE.CourseTitle, cd.INSTRUCTOR.InstructorName, Section, Meetings, TimeCodes, Term, StartDay, End, Cap, Act, Rem
FROM cd.COURSE_OFFERING
     -- JOIN COURSE_MEETING1 ON (cd.COURSE_MEETING1.CatalogID = cd.CatalogID)
     -- JOIN INSTRUCTOR1 ON (cd.INSTRUCTOR1.InstructorName = cd.InstructorName)
     -- LEFT JOIN COURSE1 ON (cd.COURSE1.CatalogID = cd.CatalogID)

 * sqlite:///Course_DW.db
0 rows affected.
(sqlite3.OperationalError) no such column: cd.COURSE.CatalogID
[SQL: INSERT INTO COURSE_OFFERING1 (CatalogID, CourseTitle, InstructorName, Section, Meetings, TimeCodes, Term, StartDay, End, Cap, Act, Rem)
SELECT DISTINCT cd.COURSE.CatalogID, cd.COURSE.CourseTitle, cd.INSTRUCTOR.InstructorName, Section, Meetings, TimeCodes, Term, StartDay, End, Cap, Act, Rem
FROM cd.COURSE_OFFERING
     -- JOIN COURSE_MEETING1 ON (cd.COURSE_MEETING1.CatalogID = cd.CatalogID)
     -- JOIN INSTRUCTOR1 ON (cd.INSTRUCTOR1.InstructorName = cd.InstructorName)
     -- LEFT JOIN COURSE1 ON (cd.COURSE1.CatalogID = cd.CatalogID)]
(Background on this error at: http://sqlalche.me/e/e3q8)


In [7]:
%%sql
DROP TABLE IF EXISTS PROGRAM1;
CREATE TABLE PROGRAM1 (
    ProgramID INTEGER PRIMARY KEY,
    ProgramCode TEXT,
    ProgramName TEXT
);

 * sqlite:///Course_DW.db
Done.
Done.


[]

In [ ]:
%%sql

DROP TABLE IF EXISTS COURSE1;
CREATE TABLE COURSE1 (
    CatalogID TEXT,
    CourseID INTEGER PRIMARY KEY,
    ProgramID INTEGER NOT NULL,
    CourseTitle TEXT NOT NULL,
    Description TEXT,
    Attributes TEXT,
    Coreqs TEXT,
    Prereqs TEXT,
    Fees TEXT,
    Credits INTEGER NOT NULL,
    FOREIGN KEY (ProgramID) REFERENCES PROGRAM (ProgramID)
);

In [ ]:
%%sql

DROP TABLE IF EXISTS COURSE_MEETING;
CREATE TABLE COURSE_MEETING (
    CourseMeetingID INTEGER PRIMARY KEY,
    CatalogID TEXT,
    Location TEXT,
    StartDateTime TEXT,
    EndDateTime TEXT,
    FOREIGN KEY (Location) REFERENCES LOCATION (Location)
);

In [44]:
%%sql
vacuum;

 * sqlite:///Course_DW.db
Done.


[]